In [4]:
print('Hello LangChain')

Hello LangChain


In [5]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
#print(OPENAI_API_KEY)

In [6]:
# prompt
prompt = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 개발자입니다.") , 
     ("human", "{input}") ]
)
print(prompt)

prompt_text = prompt.format(input="LangServe는 무엇인가요? 자세하게 설명해주세요")

input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 개발자입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


In [7]:
# Groq API를 사용하는 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    #model="mistral-saba-24b",
    temperature=0.7
)
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x000001B24FB3CC20> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001B24FB7A240> root_client=<openai.OpenAI object at 0x000001B24FB78C80> root_async_client=<openai.AsyncOpenAI object at 0x000001B24FB3DA00> model_name='meta-llama/llama-4-scout-17b-16e-instruct' temperature=0.7 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'


In [8]:
try:
    response = llm.invoke(prompt_text)
    print(type(response))
    print(response)
    print("응답:", response.content)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'langchain_core.messages.ai.AIMessage'>
content='LangServe는 LangChain이라는 프레임워크를 기반으로 구축된 오픈 소스 라이브러리입니다. LangChain은 자연어 처리(NLP) 및 생성 AI 애플리케이션을 개발하기 위한 프레임워크로, 대규모 언어 모델(LLM)을 활용하여 애플리케이션을 구축할 수 있습니다.\n\nLangServe는 LangChain을 사용하여 구축된 애플리케이션을 쉽게 배포하고 관리할 수 있도록 설계되었습니다. 주로 다음과 같은 기능을 제공합니다:\n\n1. **API 서버 생성**: LangServe를 사용하면 LangChain으로 구축된 애플리케이션을 API 서버로 쉽게 변환할 수 있습니다. 이를 통해 프론트엔드 애플리케이션 또는 다른 서비스에서 LangChain 모델을 쉽게 호출할 수 있습니다.\n\n2. **자동화된 API 엔드포인트 생성**: LangServe는 LangChain의 체인을 기반으로 API 엔드포인트를 자동으로 생성합니다. 이는 개발자가 별도로 API 엔드포인트를 구현하고 관리할 필요가 없다는 것을 의미합니다.\n\n3. **실시간 데이터 처리**: LangServe는 실시간으로 데이터를 처리하고, 자연어 입력에 대한 응답을 생성할 수 있습니다. 이는 대화형 애플리케이션 또는 실시간 데이터 분석과 같은 시나리오에서 특히 유용합니다.\n\n4. **확장성 및 성능**: LangServe는 확장성이 뛰어나고 고성능을 제공합니다. 대규모 언어 모델을 효율적으로 처리하고, 많은 양의 요청을 처리할 수 있습니다.\n\n5. **쉬운 통합**: LangServe는 다양한 데이터 소스 및 서비스와의 통합을 지원합니다. 따라서 개발자는 기존 시스템이나 데이터 소스를 LangServe 기반 애플리케이션과 쉽게 통합할 수 있습니다.\n\n6. **모니터링 및 로그 관리**: LangServe는 모니터링 및 로그 관리 기능을 제공하여, 애플리케이션의 성능을 모니터링하고 문제를 

### LCEL
* Prompt + LLM을 Chain으로 연결하기

In [9]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
    You are an expert in AI Expert. Answer the question. 
    <Question>: {input}에 대해 쉽게 반드시 한글로 설명해주세요.")
    """)                                     
prompt

PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='\n    You are an expert in AI Expert. Answer the question. \n    <Question>: {input}에 대해 쉽게 반드시 한글로 설명해주세요.")\n    ')

In [10]:
# chain 연결 (LCEL) prompt + llm 연결
chain = prompt | llm
print(type(chain))

<class 'langchain_core.runnables.base.RunnableSequence'>


### LCEL
* Prompt + LLM + OutputParser을 Chain으로 연결하기

In [11]:
from langchain_core.output_parsers import StrOutputParser

# chain 연결 (LCEL) prompt + llm + outputparser
output_parser = StrOutputParser()

chain2 = prompt | llm | output_parser
print(type(chain2))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [12]:
# chain 호출
try:
    result = chain.invoke({"input": "인공지능 모델의 학습 원리"})
    print(type(result))
    print(result.content)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'langchain_core.messages.ai.AIMessage'>
인공지능 모델의 학습 원리를 쉽게 설명드리겠습니다.

**인공지능 모델의 학습 원리**

인공지능 모델은 데이터를 통해 학습합니다. 이 과정은 사람의 뇌가 새로운 정보를 학습하는 것과 유사합니다. 모델은 주어진 데이터를 분석하고, 패턴을 찾아내며, 이를 통해 미래의 새로운 데이터에 대해 예측하거나 분류할 수 있습니다.

**단계별 학습 과정**

1. **데이터 수집**: 인공지능 모델을 학습시키기 위해 필요한 데이터를 수집합니다. 이 데이터는 문제의 유형에 따라 달라지며, 예를 들어 이미지 분류 모델을 만든다면 다양한 이미지 데이터를 수집해야 합니다.

2. **데이터 전처리**: 수집한 데이터를 모델이 학습할 수 있도록 가공하는 단계입니다. 데이터의 품질이 좋지 않은 경우, 데이터를 정제하거나 변환하여 학습에 적합한 형태로 만듭니다.

3. **모델 선택**: 학습하려는 문제의 유형에 따라 적합한 모델을 선택합니다. 예를 들어, 이미지 분류 문제라면 합성곱 신경망(CNN)과 같은 모델을 선택할 수 있습니다.

4. **모델 학습**: 선택한 모델에 데이터를 입력하여 모델이 스스로 패턴을 학습하도록 합니다. 이 과정에서는 모델이 예측한 결과와 실제 결과의 차이를 계산하는 손실 함수(loss function)를 정의하고, 이 손실을 최소화하는 방향으로 모델의 매개변수를 조정합니다.

5. **모델 평가**: 학습된 모델의 성능을 평가합니다. 이는 테스트 데이터를 사용하여 모델의 예측 성능을 측정하는 과정입니다.

6. **모델 개선**: 모델의 성능이 만족스럽지 않은 경우, 모델의 구조를 변경하거나, 하이퍼파라미터를 조정하는 등의 방법으로 모델을 개선합니다.

**핵심 개념**

- **신경망**: 인간의 뇌와 유사한 구조로, 여러 층의 노드(또는 뉴런)로 구성되어 있습니다. 각 노드는 입력값을 받아서 출력값을 생성하며, 이 과정에서 가중치와 활성화 함수를 사용합니다.

- *

In [13]:
# chain 호출
try:
    result = chain2.invoke({"input": ": LangChain의 Products(제품)는 어떤 것들이 있나요? 예를 들어 LangSmith, LangServe 같은 Product가 있어"})
    print(type(result))
    print(result)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'str'>
LangChain은 다양한 제품과 도구를 제공하여 개발자들이 인공지능(AI) 모델을 보다 쉽게 구축하고 활용할 수 있도록 지원합니다. 주요 제품에는 LangSmith, LangServe, LangChain CLI 등이 있습니다.

1. **LangSmith**: 
   - 랭스미스(LangSmith)는 랭체인(LangChain)에서 제공하는 플랫폼으로, 개발자들이 AI 기반 애플리케이션을 개발, 테스트, 배포할 수 있도록 지원합니다. 
   - 이 플랫폼은 개발 과정의 각 단계를 간소화하여, 모델의 성능을 평가하고, 프롬프트를 테스트하며, 에이전트를 시뮬레이션하는 등의 작업을 포함합니다.
   - 랭스미스를 통해 개발자들은 AI 애플리케이션의 개발 및 관리 프로세스를 보다 효율적으로 진행할 수 있습니다.

2. **LangServe**:
   - 랭서브(LangServe)는 LangChain에서 제공하는 또 다른 제품으로, 랭체인 애플리케이션의 배포를 간소화하는 API 플랫폼입니다.
   - 개발자들은 랭서브를 통해 LangChain에서 구축한 모델과 에이전트를 쉽고 빠르게 배포할 수 있습니다. 
   - 랭서브는 사용자 친화적인 인터페이스와 강력한 인프라를 제공하여, 개발자가 인프라 관리에 많은 시간을 들이지 않고도 애플리케이션 개발에 집중할 수 있도록 돕습니다.

3. **LangChain CLI**:
   - 랭체인 CLI는 명령어 기반 인터페이스를 제공하여, 개발자들이 LangChain의 다양한 기능을 명령어를 통해 손쉽게 사용할 수 있도록 합니다.
   - 이 도구를 통해 개발자들은 LangChain 생태계 내에서 다양한 작업을 보다 효율적으로 수행할 수 있습니다.

이러한 제품과 도구들은 개발자들이 LangChain을 활용하여 보다 쉽고 효율적으로 AI 애플리케이션을 개발하고 배포할 수 있도록 지원합니다.


### Runnable의 stream() 함수 호출

In [14]:
# 스트리밍 출력을 위한 요청
try:
    answer = chain2.stream({"input": "인공지능 모델의 학습 원리를 자세하게 설명해 주세요."})
    
    # 스트리밍 출력
    #print(answer)
    for token in answer:
        # 스트림에서 받은 데이터의 내용을 출력합니다. 줄바꿈 없이 이어서 출력하고, 버퍼를 즉시 비웁니다.
        print(token, end="", flush=True)
except Exception as e:
    print(f"오류 발생: {e}")


인공지능 모델의 학습 원리를 설명드리겠습니다.

**인공지능 모델의 학습이란?**

인공지능 모델의 학습은 컴퓨터가 데이터를 분석하고 학습하여, 주어진 문제에 대한 해결책을 스스로 찾아내는 과정입니다. 이 과정은 사람의 뇌가 학습하는 방식과 유사합니다.

**학습의 기본 요소**

인공지능 모델의 학습에는 다음과 같은 기본 요소가 필요합니다.

1. **데이터**: 인공지능 모델이 학습하기 위해서는 데이터가 필요합니다. 데이터는 문제에 대한 답이 될 수 있는 정보입니다.
2. **모델**: 모델은 인공지능이 학습하는 데 사용하는 수학적 알고리즘입니다.
3. **목적 함수**: 목적 함수는 모델이 최적화해야 하는 함수입니다. 목적 함수는 모델의 성능을 평가하는 데 사용됩니다.

**학습 과정**

인공지능 모델의 학습 과정은 다음과 같습니다.

1. **데이터 수집**: 데이터를 수집합니다.
2. **데이터 전처리**: 수집한 데이터를 분석하고 가공하여 모델이 학습할 수 있는 형태로 변환합니다.
3. **모델 정의**: 모델을 정의합니다. 모델은 신경망, 의사결정나무, 선형회귀 등 다양한 형태가 있습니다.
4. **목적 함수 정의**: 목적 함수를 정의합니다. 목적 함수는 모델의 성능을 평가하는 데 사용됩니다.
5. **학습**: 모델이 데이터를 학습합니다. 학습 과정에서는 모델이 데이터를 분석하고, 목적 함수를 최적화합니다.
6. **평가**: 학습된 모델의 성능을 평가합니다.

**신경망 기반 모델의 학습**

신경망 기반 모델의 학습은 다음과 같습니다.

1. **신경망 구조 정의**: 신경망의 구조를 정의합니다. 신경망은 입력층, 은닉층, 출력층으로 구성됩니다.
2. **가중치 초기화**: 신경망의 가중치를 초기화합니다.
3. **순전파**: 입력 데이터를 신경망에 입력하여 출력값을 계산합니다.
4. **역전파**: 출력값과 실제값 사이의 오차를 계산하고, 가중치를 업데이트합니다.
5. **최적화**: 가중치를 최적화하여 목적 함수를 최소화합니다.

**

### Multi Chain
* 첫번째 Chain의 출력이, 두번째 Chain의 입력이 된다.
* 두개의 Chain과 Prompt + OutputParser를 LCEL로 연결하기

In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Step 1: 사용자가 입력한 장르에 따라 영화 추천
prompt1 = ChatPromptTemplate.from_template("{genre} 장르에서 추천할 만한 한국 영화를 한 편 알려주세요.")

# Step 2: 추천된 영화의 줄거리를 요약
prompt2 = ChatPromptTemplate.from_template("{movie} 추전한 영화의 제목을 먼저 알려주시고, 줄을 바꾸어서 영화의 줄거리를 10문장으로 요약해 주세요.")

# OpenAI 모델 사용
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# 체인 1: 영화 추천 (입력: 장르 → 출력: 영화 제목)
chain1 = prompt1 | llm | StrOutputParser()

In [16]:
# 체인 2: 줄거리 요약 (입력: 영화 제목 → 출력: 줄거리)
try:
    chain2 = (
        {"movie": chain1}  # chain1의 출력을 movie 입력 변수로 전달
        | prompt2
        | llm
        | StrOutputParser()
    )

    # 실행: "SF" 장르의 영화 추천 및 줄거리 요약
    response = chain2.invoke({"genre": "액션"})
    print(response)  
except Exception as e:
    print(f"오류 발생: {e}")

### 영화 정보

*   **영화 제목**: 범죄조직과 맞서 싸운다는 점과 전직 특수요원 출신의 주인공이 등장한다는 점에서, 영화 제목은 <범죄와의 전쟁: 재영>으로 정하겠습니다.
*   **주연**: 정재영, 박희순, 이준

### 영화 줄거리 요약

대한민국의 한 작은 마을에 범죄조직이 등장합니다. 이 조직은 마을을 장악하고, 주민들은 극심한 공포에 시달립니다. 이에 맞서 마을을 구하기 위해 전직 특수요원 출신의 '재영'이 등장합니다. 재영은 강력한 범죄조직과 맞서 싸우며 마을을 구하려고 합니다. 이 영화는 한국에서 제작된 액션 영화로, 강렬한 액션과 긴박한 스토리로 구성되어 있습니다. 재영은 특수요원 시절의 경험을 바탕으로 범죄조직과의 싸움에서 우위를 차지하려 합니다. 그러나 범죄조직은 매우 강력하고, 재영은 많은 어려움을 겪습니다. 재영의 노력에도 불구하고, 범죄조직은 쉽게 물러서지 않습니다. 결국 재영은 모든 것을 걸고 범죄조직과 최후의 대결을 펼칩니다. 재영의 결단과 용기는 마을 주민들에게 큰 희망을 줍니다.


### PromptTemplate 여러개 연결하기

In [17]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 요약해서 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(model_name="ChatGPT", count=3)

# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)
combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요")
              + "\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요."
)
combined_prompt.format(model_name="ChatGPT", count=3, language="영어")

# OpenAI 모델 사용
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

chain = combined_prompt | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3, "language":"영어"})
print(response)

ChatGPT 모델의 학습 원리는 다음과 같습니다.

ChatGPT는 대규모 언어 데이터셋을 기반으로 하는 딥러닝 모델로, 주어진 문맥에 따라 다음에 올 수 있는 단어를 예측하도록 학습합니다. 이를 위해 GPT 모델은 트랜스포머 아키텍처를 사용하여 입력 문장의 각 토큰에 대한 컨텍스트를 이해하고, 그에 따라 다음 토큰을 예측합니다. 학습 과정에서는 주어진 데이터셋에 대해 모델이 예측한 토큰과 실제 토큰 간의 차이를 최소화하도록 모델을 업데이트합니다.

ChatGPT 모델의 장점은 다음과 같습니다.

* 자연스러운 대화 생성: ChatGPT는 대화형 AI 모델로, 사용자와 자연스러운 대화를 나눌 수 있습니다.
* 다양한 주제 이해: ChatGPT는 다양한 주제에 대해 이해하고 답변할 수 있습니다.
* 높은 정확도: ChatGPT는 높은 정확도로 답변을 생성할 수 있습니다.

ChatGPT 모델과 비슷한 AI 모델은 다음과 있습니다.

* LLaMA
* BERT
* RoBERTa
* transformer-XL
* XLNet

특히, LLaMA는 Meta에서 개발한 대규모 언어 모델로, ChatGPT와 유사한 아키텍처와 학습 방법을 사용합니다.


In [18]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 요약해서 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

questions = [
    {"model_name": "GPT-4", "count": 2},
    {"model_name": "Gemma", "count": 3},
    {"model_name": "llama-4", "count": 4},
]

# 여러 개의 프롬프트를 미리 생성
formatted_prompts = [prompt_template.format(**q) for q in questions]
print(formatted_prompts)  # 미리 생성된 질문 목록 확인

['GPT-4 모델의 학습 원리를 2 문장으로 요약해서 한국어로 답변해 주세요.', 'Gemma 모델의 학습 원리를 3 문장으로 요약해서 한국어로 답변해 주세요.', 'llama-4 모델의 학습 원리를 4 문장으로 요약해서 한국어로 답변해 주세요.']


In [19]:
# OpenAI 모델 사용
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

for prompt in formatted_prompts:
    response = llm.invoke(prompt) #AIMessage
    print(response.content)

GPT-4 모델은 대규모 데이터셋을 기반으로 하는 딥러닝 알고리즘을 사용하여 학습되며, 자연어 처리 작업에 특화된 트랜스포머 아키텍처를 활용합니다. 이 모델은 주어진 문맥을 이해하고 다음에 올 가능성이 높은 단어를 예측하도록 학습되며, 이를 통해 다양한 자연어 처리 작업에 활용될 수 있습니다.
Gemma 모델은 주어진 문맥을 기반으로 다음 토큰을 예측하는 자기 지도 학습 방식을 사용합니다. 대규모 텍스트 데이터 세트에서 패턴과 관계를 학습하여 언어 구조에 대한 이해를 발전시킵니다. 이 학습을 통해 Gemma는 사람과 유사한 텍스트를 생성하고 다양한 언어 관련 작업을 효율적으로 수행할 수 있습니다.
llama-4 모델은 메타에서 개발한 대규모 언어 모델입니다. 이 모델은 방대한 양의 텍스트 데이터를 바탕으로 학습되며, 이를 통해 자연어 처리 능력을 습득합니다. 학습 과정에서 모델은 주어진 문맥에서 다음에 올 수 있는 단어를 예측하도록 훈련되며, 이 과정을 통해 언어의 패턴과 구조를 학습합니다. 이를 통해 llama-4 모델은 다양한 자연어 처리 작업에 활용될 수 있습니다.


### ChatPromptTemplate 
* SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate 사용

In [20]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate
)
from langchain_openai import ChatOpenAI

# 개별 메시지 템플릿 정의
system_message = SystemMessagePromptTemplate.from_template(
    "당신은 {topic} 전문가입니다. 명확하고 자세하게 한국어로 설명해 주세요."
)
user_message = HumanMessagePromptTemplate.from_template(
    "{question}"
)
ai_message = AIMessagePromptTemplate.from_template(
    "This is an example answer about {topic}."
)

# ChatPromptTemplate로 메시지들을 묶기
chat_prompt = ChatPromptTemplate.from_messages([
    system_message,
    user_message,
    ai_message
])

# 메시지 생성
messages = chat_prompt.format_messages(topic="AI", question="딥러닝은 무엇인가요?")

# LLM 호출
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
response = llm.invoke(messages)

# 결과 출력
print(response.content)

### FewShotPromptTemplate
* 예시를 제공 프롬프트

In [21]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI

examples = [
    {
        "input": "뉴턴의 운동 법칙을 요약해 주세요.",
        "output": """### 뉴턴의 운동 법칙
1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.
2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.
3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다."""
    },
    {
        "input": "지구의 대기 구성 요소를 알려주세요.",
        "output": """### 지구 대기의 구성
- **질소 (78%)**: 대기의 대부분을 차지합니다.
- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.
- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.
- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다."""
    }
]

# 예제 프롬프트 템플릿
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# FewShotChatMessagePromptTemplate 적용
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 구성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 초등학생도 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# 모델 생성 및 체인 구성
#model = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

chain = final_prompt | model

# 테스트 실행
result = chain.invoke({"input": "태양계의 행성들을 간략히 정리해 주세요."})
#result = chain.invoke({"input": "양자 얽힘이 무엇인가요?"})
print(result.content)

### 태양계의 행성
1. **수성**: 태양과 가장 가까운 행성으로, 표면이 매우 뜨겁고 차갑습니다.
2. **금성**: 두꺼운 대기로 인해 매우 뜨겁고 밝습니다.
3. **지구**: 생명체가 살고 있는 유일한 행성입니다.
4. **화성**: 붉은 행성으로, 로봇 탐사선이 많이 방문했습니다.
5. **목성**: 태양계에서 가장 큰 행성으로 가스 행성입니다.
6. **토성**: 아름다운 고리를 가진 가스 행성입니다.
7. **천왕성**: 자전축이 기울어져 있어 극단적인 계절 변화를 경험합니다.
8. **해왕성**: 태양계에서 가장 먼 행성으로, 매우 차갑습니다.


### PartialPromptTemplate
* 프롬프트의 입력 값에 함수 호출 이나 외부 API를 호출한 동적인 값을 대입할 수 있음

In [22]:
from datetime import datetime
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month
    
    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# 프롬프트 템플릿 정의 (부분 변수 적용)
prompt = PromptTemplate(
    template="{season}에 일어나는 대표적인 지구과학 현상은 {phenomenon}입니다.",
    input_variables=["phenomenon"],  # 사용자 입력 필요
    partial_variables={"season": get_current_season("south")}  # 동적으로 계절 값 할당
)

# OpenAI 모델 초기화
#model = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.5
)

# 특정 계절의 현상 질의
query = prompt.format(phenomenon="태풍 발생")  # '태풍 발생'은 여름과 관련됨
result = model.invoke(query)


# 결과 출력
print(f"🔹 프롬프트: {query}")
print(f"🔹 모델 응답: {result.content}")

🔹 프롬프트: 겨울에 일어나는 대표적인 지구과학 현상은 태풍 발생입니다.
🔹 모델 응답: 겨울에 태풍이 발생하는 경우는 드문 일입니다. 태풍은 일반적으로 여름과 초가을에 발생합니다. 겨울에는 태풍이 발생하지 않는 것은 아니지만, 극히 드문 현상입니다.


In [23]:
import requests
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 실시간 환율을 가져오는 함수
def get_exchange_rate():
    response = requests.get("https://api.exchangerate-api.com/v4/latest/USD")
    data = response.json()
    return f"1달러 = {data['rates']['KRW']}원"

# {info} 변수에 API에서 받은 환율 정보를 동적으로 반영

prompt = PromptTemplate(
    template="현재 {info} 기준으로 환율 정보를 알려드립니다. 현재 환율을 기준으로 한국경제 미치는 영향 및 향후에 환율의 예상값에 대한 분석을 제공해 주세요.",
    input_variables=[],  # 사용자 입력 없음
    partial_variables={"info": get_exchange_rate()}  # API에서 가져온 데이터 자동 반영
)
print(prompt)

input_variables=[] input_types={} partial_variables={'info': '1달러 = 1365.14원'} template='현재 {info} 기준으로 환율 정보를 알려드립니다. 현재 환율을 기준으로 한국경제 미치는 영향 및 향후에 환율의 예상값에 대한 분석을 제공해 주세요.'


In [25]:

# LLM 모델 설정 (GPT-4o-mini 사용)
model = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

# OpenAI 모델 초기화
#model = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.5
)

# 모델에 프롬프트 전달 및 응답 받기
response = model.invoke(prompt.format())

# 결과 출력
print("🔹 프롬프트:", prompt.format())
print("🔹 모델 응답:", response.content)

🔹 프롬프트: 현재 1달러 = 1365.14원 기준으로 환율 정보를 알려드립니다. 현재 환율을 기준으로 한국경제 미치는 영향 및 향후에 환율의 예상값에 대한 분석을 제공해 주세요.
🔹 모델 응답: ## 현재 환율: 1달러 = 1365.14원

### 한국 경제에 미치는 영향:

1. **수출 증가**: 높은 환율은 한국의 수출을 촉진할 수 있습니다. 
2. **물가 상승**: 수입 물가가 상승하여 국내 물가에도 영향을 미칠 수 있습니다. 
3. **기업의 수익성**: 수출 비중이 높은 기업들은 환율 상승으로 인해 수익성이 개선될 수 있습니다. 
4. **소비자 심리**: 환율 상승으로 인해 소비자 물가가 상승하면 소비자들의 구매력이 감소할 수 있습니다.

### 향후 환율 예상:

1. **글로벌 경제 상황**: 미국의 경제 상황, 금리 변동, 글로벌 무역 상황 등이 환율에 영향을 미칠 수 있습니다.
2. **한국 경제 상황**: 한국의 경제 성장률, 물가 상승률, 금리 변동 등이 환율에 영향을 미칠 수 있습니다.
3. **원화 가치**: 원화의 가치 변동이 환율에 영향을 미칠 수 있습니다.

환율은 다양한 요인에 의해 변동하므로 정확한 예측은 어렵습니다.
